In [4]:
import faiss
import numpy as np
import pickle
import time

N = 11903
k = 8

nlist = 100  # Numero de clusters
m = 16        # Cantidad de subespacios en PQ

with open("feature_spotify.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features = pickle.load(file)

features_flat = np.vstack([np.array(f, dtype='float32') for f in features[:N] if len(f[0]) == 1280])

quantizer = faiss.IndexFlatL2(1280)  # Quantizador base        # distancia euclidiana
index_ivfpq = faiss.IndexIVFPQ(quantizer, 1280, nlist, m, 8)

# Indexacion
start_time = time.time()
index_ivfpq.train(features_flat)
index_ivfpq.add(features_flat)
indexation_time = time.time() - start_time
print(f"Tiempo de indexación (IVFPQ CPU): {indexation_time:.2f} segundos")

# Busqueda K-NN
def knn_search_ivfpq(query_index, k):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    D, I = index_ivfpq.search(query_vector, k)
    search_time = time.time() - start_time
    print(f"Búsqueda k-NN:")
    print("Distancias:", D[0])
    print("Índices:", I[0])
    print(f"Tiempo de búsqueda k-NN: {search_time:.2f} segundos")
    return search_time

# Busqueda por rango
def range_search_ivfpq(query_index, radius):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    lims, D, I = index_ivfpq.range_search(query_vector, radius)
    range_search_time = time.time() - start_time
    print(f"Búsqueda por rango:")
    print("Distancias:", D)
    print("Índices:", I)
    # for i in range(lims[0], lims[1]):
    #     print(f"Índice: {I[i]}, Distancia: {D[i]}")
    print(f"Tiempo de búsqueda por rango: {range_search_time:.2f} segundos")
    return range_search_time

query_index = 100
rad = 5.5*10**6
# rad = 50000000

# Ejecutar búsqueda K-NN
print()
knn_search_ivfpq(query_index, k)

# Ejecutar búsqueda por rango
print()
range_search_ivfpq(query_index, rad)


Tiempo de indexación (IVFPQ CPU): 37.36 segundos

Búsqueda k-NN:
Distancias: [4069372.5 5115957.5 5126740.5 5139553.  5145473.  5156438.  5166973.
 5188542. ]
Índices: [   100  59842 204240   4980  11380  27880 113460  71320]
Tiempo de búsqueda k-NN: 0.00 segundos

Búsqueda por rango:
Distancias: [4069372.5 5283658.  5139553.  5462076.5 5145473.  5442923.  5218200.
 5156438.  5498279.  5306655.5 5446664.5 5409887.5 5388706.5 5115957.5
 5491899.  5431830.  5188542.  5494858.  5478366.  5385044.5 5392237.
 5488197.  5402395.5 5402854.  5490252.5 5166973.  5465943.  5230621.
 5311259.  5486010.  5459155.  5448962.5 5418455.5 5436960.  5469574.5
 5435938.5 5438075.  5491748.5 5126740.5 5441240.5 5347610.  5492811.
 5214279.  5367422.  5367422.  5348628.5 5411836. ]
Índices: [   100   3200   4980   8800  11380  12100  25262  27880  33480  37160
  42800  48320  59840  59842  61880  66500  71320  74660  78760  83840
  86720  94880  99060 102120 102740 113460 120700 132600 133000 133040
 14932

0.0